In [ ]:
from datasets import load_dataset, load_dataset_builder
import pandas as pd

import os
import time
import numpy as np
from pathlib import Path # makes paths more readable

In [3]:
# load foursquare open places dataset builder to inspect it
ds_builder = load_dataset_builder("foursquare/fsq-os-places", "places")

Resolving data files:   0%|          | 0/100 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/100 [00:00<?, ?it/s]

In [4]:
# inspect dataset features
ds_builder.info.features

{'fsq_place_id': Value('string'),
 'name': Value('string'),
 'latitude': Value('float64'),
 'longitude': Value('float64'),
 'address': Value('string'),
 'locality': Value('string'),
 'region': Value('string'),
 'postcode': Value('string'),
 'admin_region': Value('string'),
 'post_town': Value('string'),
 'po_box': Value('string'),
 'country': Value('string'),
 'date_created': Value('string'),
 'date_refreshed': Value('string'),
 'date_closed': Value('string'),
 'tel': Value('string'),
 'website': Value('string'),
 'email': Value('string'),
 'facebook_id': Value('int64'),
 'instagram': Value('string'),
 'twitter': Value('string'),
 'fsq_category_ids': List(Value('string')),
 'fsq_category_labels': List(Value('string')),
 'placemaker_url': Value('string'),
 'unresolved_flags': List(Value('string')),
 'geom': Value('binary'),
 'bbox': {'xmin': Value('float64'),
  'ymin': Value('float64'),
  'xmax': Value('float64'),
  'ymax': Value('float64')}}

In [5]:
# us and canada ('country' column)
countries_include = ['US', 'CA']

# countries to exclude ('country' column)
countries_exclude = [
    "AF", "AX", "AL", "DZ", "AS", "AD", "AO", "AI", "AQ", "AG", "AR", "AM", "AW", "AU", "AT", "AZ",
    "BS", "BH", "BD", "BB", "BY", "BE", "BZ", "BJ", "BM", "BT", "BO", "BQ", "BA", "BW", "BV", "BR",
    "IO", "BN", "BG", "BF", "BI", "CV", "KH", "CM", "KY", "CF", "TD", "CL", "CN", "CX", "CC", "CO",
    "KM", "CG", "CD", "CK", "CR", "CI", "HR", "CU", "CW", "CY", "CZ", "DK", "DJ", "DM", "DO", "EC",
    "EG", "SV", "GQ", "ER", "EE", "SZ", "ET", "FK", "FO", "FJ", "FI", "FR", "GF", "PF", "TF", "GA",
    "GM", "GE", "DE", "GH", "GI", "GR", "GL", "GD", "GP", "GU", "GT", "GG", "GN", "GW", "GY", "HT",
    "HM", "VA", "HN", "HK", "HU", "IS", "IN", "ID", "IR", "IQ", "IE", "IM", "IL", "IT", "JM", "JP",
    "JE", "JO", "KZ", "KE", "KI", "KP", "KR", "KW", "KG", "LA", "LV", "LB", "LS", "LR", "LY", "LI",
    "LT", "LU", "MO", "MG", "MW", "MY", "MV", "ML", "MT", "MH", "MQ", "MR", "MU", "YT", "MX", "FM",
    "MD", "MC", "MN", "ME", "MS", "MA", "MZ", "MM", "NA", "NR", "NP", "NL", "NC", "NZ", "NI", "NE",
    "NG", "NU", "NF", "MK", "MP", "NO", "OM", "PK", "PW", "PS", "PA", "PG", "PY", "PE", "PH", "PN",
    "PL", "PT", "PR", "QA", "RE", "RO", "RU", "RW", "BL", "SH", "KN", "LC", "MF", "PM", "VC", "WS",
    "SM", "ST", "SA", "SN", "RS", "SC", "SL", "SG", "SX", "SK", "SI", "SB", "SO", "ZA", "GS", "SS",
    "ES", "LK", "SD", "SR", "SJ", "SE", "CH", "SY", "TW", "TJ", "TZ", "TH", "TL", "TG", "TK", "TO",
    "TT", "TN", "TR", "TM", "TC", "TV", "UG", "UA", "AE", "GB", "UY", "UZ", "VU", "VE", "VN", "VG",
    "VI", "WF", "EH", "YE", "ZM", "ZW", "UM",
]

# us states ('region' column)
us_states_include = [
    "AL", "AK", "AZ", "AR", "AS", "CA", "CO", "CT", "DE", "DC", "FL", "GA", "GU", "HI", "ID", "IL", "IN",
    "IA", "KS", "KY", "LA", "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", 
    "NY", "NC", "ND", "MP", "OH", "OK", "OR", "PA", "PR", "RI", "SC", "SD", "TN", "TX", "TT", "UT", "VT", 
    "VA", "VI", "WA", "WV", "WI", "WY",
]

# canadian provinces ('region' column)
ca_provinces_include = [
    "NL", "PE", "NS", "NB", "QC", "ON", "MB", "SK", "AB", "BC", "YT", "NT", "NU",
]



In [ ]:
# fsq chinese restaurant category ids
# after filtering was applied (below), some restaurants are just labeled "chinese" with no subcategory
# this may not have been the case before filtering, but hopefully that doesn't matter for this project


chinese_category_ids = [
    "4bf58dd8d48988d145941735",  # Chinese Restaurant (main category)
    "52af3a723cf9994f4e043bec",  # Beijing Restaurant
    "52af3a7c3cf9994f4e043bed",  # Cantonese Restaurant
    "58daa1558bbb0b01f18ec1d3",  # Cha Chaan Teng
    "52af3a673cf9994f4e043beb",  # Chinese Aristocrat Restaurant
    "52af3a903cf9994f4e043bee",  # Chinese Breakfast Restaurant
    "4bf58dd8d48988d1f5931735",  # Dim Sum Restaurant
    "52af3a9f3cf9994f4e043bef",  # Dongbei Restaurant
    "52af3aaa3cf9994f4e043bf0",  # Fujian Restaurant
    "52af3ab53cf9994f4e043bf1",  # Guizhou Restaurant
    "52af3abe3cf9994f4e043bf2",  # Hainan Restaurant
    "52af3ac83cf9994f4e043bf3",  # Hakka Restaurant
    "52af3ad23cf9994f4e043bf4",  # Henan Restaurant
    "52af3add3cf9994f4e043bf5",  # Hong Kong Restaurant
    "52af3af23cf9994f4e043bf7",  # Huaiyang Restaurant
    "52af3ae63cf9994f4e043bf6",  # Hubei Restaurant
    "52af3afc3cf9994f4e043bf8",  # Hunan Restaurant
    "52af3b053cf9994f4e043bf9",  # Imperial Restaurant
    "52af3b213cf9994f4e043bfa",  # Jiangsu Restaurant
    "52af3b293cf9994f4e043bfb",  # Jiangxi Restaurant
    "52af3b343cf9994f4e043bfc",  # Macanese Restaurant
    "52af3b3b3cf9994f4e043bfd",  # Manchu Restaurant
    "52af3b463cf9994f4e043bfe",  # Peking Duck Restaurant
    "52af3b633cf9994f4e043c01",  # Shaanxi Restaurant
    "52af3b513cf9994f4e043bff",  # Shandong Restaurant
    "52af3b593cf9994f4e043c00",  # Shanghai Restaurant
    "52af3b6e3cf9994f4e043c02",  # Shanxi Restaurant
    "52af3b773cf9994f4e043c03",  # Szechuan Restaurant
    "52af3b813cf9994f4e043c04",  # Taiwanese Restaurant
    "52af3b893cf9994f4e043c05",  # Tianjin Restaurant
    "52af3b913cf9994f4e043c06",  # Xinjiang Restaurant
    "52af3b9a3cf9994f4e043c07"   # Yunnan Restaurant
]


In [ ]:
# load full(!!!!!) dataset (11GB)
dataset = load_dataset("foursquare/fsq-os-places", "places", split="train")

Resolving data files:   0%|          | 0/100 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/100 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/88 [00:00<?, ?it/s]

In [ ]:
# extract all the chinese restaurants in us and canada

# based on country (country code) or region (state/province) variables
# include chinese restaurants with null country or region just in case

# Create output directory
output_dir = Path("chinese_restaurants_filtered")
output_dir.mkdir(exist_ok=True)

# Clear any existing files
for f in output_dir.glob("*.csv"):
    f.unlink()

batch_size = 500000
total_chinese_restaurants = 0

print("Starting batch processing with filtering...")

def is_chinese_restaurant(category_ids):
    """Check if any category ID matches Chinese restaurant categories"""
    try:
        # Check for null values
        if pd.isna(category_ids) or category_ids is None:
            return False
        
        # apparently 'fsq-category-ids' holds numpy arrays, not lists
        # Convert numpy arrays to lists
        if isinstance(category_ids, np.ndarray):
            category_ids = category_ids.tolist()
        
        # Check if it's a list and contains Chinese restaurant categories
        if isinstance(category_ids, list):
            return any(cat in chinese_category_ids for cat in category_ids)
        return False
    except:
        # Fallback: if anything goes wrong, return False
        return False

def is_us_ca_location(country, region):
    # Check if location is in US/Canada, handling nulls properly
    # If both country and region are null, return True (we'll include these)
    if pd.isna(country) and pd.isna(region):
        return True
    
    # If country is in our include list
    if not pd.isna(country) and country in countries_include:
        return True
    
    # If country is in exclude list, return False
    if not pd.isna(country) and country in countries_exclude:
        return False
    
    # If region is a US state or Canadian province
    if not pd.isna(region):
        if region in us_states_include or region in ca_provinces_include:
            return True
    
    # If we have country but it's not in include/exclude lists, be conservative
    if not pd.isna(country):
        return False
    
    # If we only have region and it's not US/CA, return False
    if not pd.isna(region):
        return False
    
    # Default case: include if we can't determine (null country/region)
    return True

try:
    start_time = time.time()
    
    for i, df_batch in enumerate(dataset.to_pandas(batch_size=batch_size, batched=True)):
        print(f"Processing batch {i+1} with {len(df_batch)} rows...")
        
        if 'country' in df_batch.columns and 'fsq_category_ids' in df_batch.columns:
            # Get region column if it exists
            region_col = 'region' if 'region' in df_batch.columns else None
            
            # Apply location filtering
            if region_col:
                location_mask = df_batch.apply(
                    lambda row: is_us_ca_location(row['country'], row[region_col]), 
                    axis=1
                )
            else:
                location_mask = df_batch['country'].apply(
                    lambda x: is_us_ca_location(x, None)
                )
            
            location_filtered = df_batch[location_mask]
            print(f"  → {len(location_filtered)} rows after location filtering")
            
            # Apply Chinese restaurant filtering
            chinese_mask = location_filtered['fsq_category_ids'].apply(is_chinese_restaurant)
            filtered_batch = location_filtered[chinese_mask]
            
            if len(filtered_batch) > 0:
                output_file = output_dir / f"chinese_restaurants_batch_{i}.csv"
                filtered_batch.to_csv(output_file, index=False)
                total_chinese_restaurants += len(filtered_batch)
                print(f"  → Found {len(filtered_batch)} Chinese restaurants")
            else:
                print(f"  → No Chinese restaurants in this batch")
        else:
            print(f"  → Skipping batch - missing required columns")
            print(f"  → Available columns: {list(df_batch.columns)}")
        
        if (i + 1) % 10 == 0:
            elapsed = time.time() - start_time
            print(f"Completed {i + 1} batches in {elapsed:.1f} seconds")
            print(f"Total Chinese restaurants found so far: {total_chinese_restaurants}")
            
except Exception as e:
    print(f"Error processing batch {i + 1}: {e}")
    import traceback
    traceback.print_exc()

print(f"Processing complete. Total Chinese restaurants found: {total_chinese_restaurants}")


Starting corrected batch processing with proper filtering...
Processing batch 1 with 500000 rows...
  → 431893 rows after location filtering
  → Found 1469 Chinese restaurants
Processing batch 2 with 500000 rows...
  → 499999 rows after location filtering
  → Found 2064 Chinese restaurants
Processing batch 3 with 500000 rows...
  → 499998 rows after location filtering
  → Found 1015 Chinese restaurants
Processing batch 4 with 500000 rows...
  → 500000 rows after location filtering
  → Found 2495 Chinese restaurants
Processing batch 5 with 500000 rows...
  → 397211 rows after location filtering
  → Found 901 Chinese restaurants
Processing batch 6 with 500000 rows...
  → 499967 rows after location filtering
  → Found 1227 Chinese restaurants
Processing batch 7 with 500000 rows...
  → 67232 rows after location filtering
  → Found 109 Chinese restaurants
Processing batch 8 with 500000 rows...
  → 0 rows after location filtering
  → No Chinese restaurants in this batch
Processing batch 9 wi

In [19]:
# Combine all the CSV files into one

csv_files = sorted(Path(output_dir).glob("chinese_restaurants_batch_*.csv"))
dfs = []
for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    dfs.append(df)

if dfs:
    combined_df = pd.concat(dfs, ignore_index=True)
    combined_output_file = Path(output_dir) / "chinese_restaurants_combined.csv"
    combined_df.to_csv(combined_output_file, index=False)
    print(f"Combined {len(csv_files)} CSV files into {combined_output_file}")
    print(f"Total rows in combined file: {len(combined_df)}")
else:
    print("No batch CSV files found to combine.")


Combined 61 CSV files into chinese_restaurants_filtered/chinese_restaurants_combined.csv
Total rows in combined file: 64361


In [22]:
chinese_restaurants = pd.read_csv(output_dir / "chinese_restaurants_combined.csv")
chinese_restaurants.head()

,fsq_place_id,name,latitude,longitude,address,locality,region,postcode,admin_region,post_town,...,email,facebook_id,instagram,twitter,fsq_category_ids,fsq_category_labels,placemaker_url,unresolved_flags,geom,bbox
0,4c8c4e92cf3ea1434a7af451,Lee's Chinese,39.444009,-123.804738,154 E Redwood Ave,Fort Bragg,CA,95437.0,NaN,NaN,...,info@bbbemail.org,1.626084e+14,NaN,NaN,['4bf58dd8d48988d145941735'],['Dining and Drinking > Restaurant > Asian Res...,https://foursquare.com/placemakers/review-plac...,NaN,b'\x00\x00\x00\x00\x01\xc0^\xf3\x80\xd5\xb0\xd...,"{'xmin': -123.80473844785541, 'ymin': 39.44400..."
1,4f442d5819836ed00192b620,China Express,39.433506,-123.805474,660 S Main St,Fort Bragg,CA,95437.0,NaN,NaN,...,NaN,NaN,NaN,NaN,['4bf58dd8d48988d145941735'],['Dining and Drinking > Restaurant > Asian Res...,https://foursquare.com/placemakers/review-plac...,NaN,b'\x00\x00\x00\x00\x01\xc0^\xf3\x8c\xe3\xdd\xf...,"{'xmin': -123.80547424960108, 'ymin': 39.43350..."
2,a194a71384e241d278559b60,Hill House Restaurant,39.310533,-123.798360,10701 Palette Dr,Mendocino,CA,95460.0,NaN,NaN,...,frontdesk@mendocinohotels.com,NaN,NaN,NaN,['4bf58dd8d48988d145941735'],['Dining and Drinking > Restaurant > Asian Res...,https://foursquare.com/placemakers/review-plac...,NaN,b'\x00\x00\x00\x00\x01\xc0^\xf3\x18T\x8a\x9b\x...,"{'xmin': -123.79836, 'ymin': 39.310533, 'xmax'..."
3,5dffd431f4a5f60008cfa1ce,Panda Express,39.130484,-123.198543,1236 Airport Park Blvd,Ukiah,CA,95482.0,NaN,NaN,...,NaN,NaN,NaN,NaN,['4bf58dd8d48988d145941735'],['Dining and Drinking > Restaurant > Asian Res...,https://foursquare.com/placemakers/review-plac...,NaN,b'\x00\x00\x00\x00\x01\xc0^\xcc\xb4\xed|X;@C\x...,"{'xmin': -123.19854294913323, 'ymin': 39.13048..."
4,4f32482419836c91c7c8471b,New Dragon,39.140123,-123.205708,765 S State St,Ukiah,CA,95482.0,NaN,NaN,...,NaN,1.115681e+14,NaN,NaN,['4bf58dd8d48988d145941735'],['Dining and Drinking > Restaurant > Asian Res...,https://foursquare.com/placemakers/review-plac...,NaN,b'\x00\x00\x00\x00\x01\xc0^\xcd*Pt\xa0@@C\x91\...,"{'xmin': -123.20570765866523, 'ymin': 39.14012..."


In [28]:
chinese_restaurants.shape

(64361, 27)

In [26]:
chinese_restaurants.columns

Index(['fsq_place_id', 'name', 'latitude', 'longitude', 'address', 'locality',
       'region', 'postcode', 'admin_region', 'post_town', 'po_box', 'country',
       'date_created', 'date_refreshed', 'date_closed', 'tel', 'website',
       'email', 'facebook_id', 'instagram', 'twitter', 'fsq_category_ids',
       'fsq_category_labels', 'placemaker_url', 'unresolved_flags', 'geom',
       'bbox'],
      dtype='object')

In [ ]:
# find all the rows with no country listed

country_null = chinese_restaurants[chinese_restaurants['country'].isna()]
print(country_null.shape)
country_null.head()

(13, 27)


,fsq_place_id,name,latitude,longitude,address,locality,region,postcode,admin_region,post_town,...,email,facebook_id,instagram,twitter,fsq_category_ids,fsq_category_labels,placemaker_url,unresolved_flags,geom,bbox
5180,57ba8b46498ed8856ca33475,Great Wei Restaurant,2.195557,102.237099,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,['4bf58dd8d48988d145941735'],['Dining and Drinking > Restaurant > Asian Res...,https://foursquare.com/placemakers/review-plac...,NaN,"b'\x00\x00\x00\x00\x01@Y\x8f,\xa1H\xba\x84@\x0...","{'xmin': 102.237099, 'ymin': 2.195557, 'xmax':..."
7661,57c5e9ca498eea2cadb12bc1,RoomFree,43.459380,39.900918,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,['4bf58dd8d48988d145941735'],['Dining and Drinking > Restaurant > Asian Res...,https://foursquare.com/placemakers/review-plac...,NaN,b'\x00\x00\x00\x00\x01@C\xf3QG\xf10Y@E\xba\xcc...,"{'xmin': 39.900918, 'ymin': 43.45938, 'xmax': ..."
7662,57c5e96d498ecbc36caea471,бар вок,43.458568,39.901039,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,['4bf58dd8d48988d145941735'],['Dining and Drinking > Restaurant > Asian Res...,https://foursquare.com/placemakers/review-plac...,NaN,b'\x00\x00\x00\x00\x01@C\xf3U>\xf6\xb5\xd4@E\x...,"{'xmin': 39.901039, 'ymin': 43.458568, 'xmax':..."
8001,57bada85498e61797111af4a,эээ,59.870010,29.855366,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,['4bf58dd8d48988d145941735'],['Dining and Drinking > Restaurant > Asian Res...,https://foursquare.com/placemakers/review-plac...,NaN,b'\x00\x00\x00\x00\x01@=\xda\xf9D$\x1c?@M\xef\...,"{'xmin': 29.855366, 'ymin': 59.87001, 'xmax': ..."
8002,57badab2498e4fd6c6a3d773,Япония,59.869533,29.854272,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,['4bf58dd8d48988d145941735'],['Dining and Drinking > Restaurant > Asian Res...,https://foursquare.com/placemakers/review-plac...,NaN,b'\x00\x00\x00\x00\x01@=\xda\xb1\x91\xdd\xe3v@...,"{'xmin': 29.854272, 'ymin': 59.869533, 'xmax':..."


In [ ]:
# remove rows with no country listed (only 13 rows, not gonna bother looking into them)
chinese_restaurants = chinese_restaurants[chinese_restaurants['country'].notna()]
chinese_restaurants.shape

# save new csv
chinese_restaurants.to_csv(output_dir / "chinese_restaurants_cleaned.csv", index=False)


In [ ]:
# find all the rows with no region (state or province) listed
# none found, but everything has lat/long coords so it's fine (see below)

region_null = chinese_restaurants[chinese_restaurants['region'].isna()]
print(region_null.shape)
region_null.head()

(1640, 27)


,fsq_place_id,name,latitude,longitude,address,locality,region,postcode,admin_region,post_town,...,email,facebook_id,instagram,twitter,fsq_category_ids,fsq_category_labels,placemaker_url,unresolved_flags,geom,bbox
26,4bd881f40b779c74b95507a0,Fu Hing,39.520284,-122.193704,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,['4bf58dd8d48988d145941735'],['Dining and Drinking > Restaurant > Asian Res...,https://foursquare.com/placemakers/review-plac...,NaN,b'\x00\x00\x00\x00\x01\xc0^\x8ce\xa5\x17\nO@C\...,"{'xmin': -122.19370391130472, 'ymin': 39.52028..."
43,4f5bb94ee4b01f8f4c30222c,Rice Wok,39.761965,-121.823380,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,['4bf58dd8d48988d145941735'],['Dining and Drinking > Restaurant > Asian Res...,https://foursquare.com/placemakers/review-plac...,NaN,b'\x00\x00\x00\x00\x01\xc0^t\xb2@\x9c\xde\xb0@...,"{'xmin': -121.82337966269802, 'ymin': 39.76196..."
237,4f6b7272e4b0d8154c2fb5ef,Manchu Wok,38.269847,-121.948936,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,['4bf58dd8d48988d145941735'],['Dining and Drinking > Restaurant > Asian Res...,https://foursquare.com/placemakers/review-plac...,NaN,"b'\x00\x00\x00\x00\x01\xc0^|\xbb^\xce[\xfb@C""\...","{'xmin': -121.94893617775226, 'ymin': 38.26984..."
391,4e79745852b1fc6a4ae6a494,MR. NOODLE,37.973042,-122.339570,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,['4bf58dd8d48988d145941735'],['Dining and Drinking > Restaurant > Asian Res...,https://foursquare.com/placemakers/review-plac...,NaN,"b'\x00\x00\x00\x00\x01\xc0^\x95\xbb\x83\x03*,@...","{'xmin': -122.33956981, 'ymin': 37.97304232, '..."
488,4d55bdd1a05c3704ca8dbc87,Emmo's Kitchen,37.851761,-122.273421,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,['4bf58dd8d48988d145941735'],['Dining and Drinking > Restaurant > Asian Res...,https://foursquare.com/placemakers/review-plac...,NaN,b'\x00\x00\x00\x00\x01\xc0^\x91\x7f\xbb\xa8DB@...,"{'xmin': -122.27342120582856, 'ymin': 37.85176..."


In [ ]:
# count rows with no lat/long coords

print(len(chinese_restaurants[chinese_restaurants['latitude'].isna() | chinese_restaurants['longitude'].isna()]))

# answer is 0 rows, yay!

0


In [ ]:
# count rows where 'date_closed' is null

open_restaurants = len(chinese_restaurants[chinese_restaurants['date_closed'].isna()])
print(f'open restaurants: {open_restaurants}')

open restaurants: 50712


In [42]:
total_restaurants = len(chinese_restaurants)
open_restaurants = len(chinese_restaurants[chinese_restaurants['date_closed'].isna()])

print(f'closed restaurants: {total_restaurants - open_restaurants}')

closed restaurants: 13636


In [ ]:
# check for duplicate rows by 'fsq_place_id'
# no duplicates found!

duplicate_ids = chinese_restaurants[chinese_restaurants.duplicated(subset=['fsq_place_id'])]
print(duplicate_ids.shape)
duplicate_ids.head()

(0, 27)


,fsq_place_id,name,latitude,longitude,address,locality,region,postcode,admin_region,post_town,...,email,facebook_id,instagram,twitter,fsq_category_ids,fsq_category_labels,placemaker_url,unresolved_flags,geom,bbox


In [49]:
# count restaurants by country and region

counts_by_country = chinese_restaurants['country'].value_counts()
counts_by_country

country
US    57536
CA     6812
Name: count, dtype: int64

In [81]:
counts_by_region = chinese_restaurants['region'].value_counts()
counts_by_region

region
CA    8578
NY    7056
TX    3540
FL    3522
ON    2833
PA    2693
NJ    2407
IL    2182
GA    1818
OH    1542
VA    1519
MA    1456
BC    1438
NC    1432
MI    1229
MD    1162
MO    1122
WA    1064
IN     815
SC     810
AZ     804
CT     803
TN     757
CO     710
WI     699
MN     673
AB     670
OR     656
AL     633
LA     596
NV     582
OK     530
QC     503
KY     484
IA     427
KS     424
HI     390
UT     384
AR     318
MS     294
NH     242
RI     234
ME     234
NM     231
NE     220
WV     204
ID     203
DE     188
MB     177
SK     174
DC     167
NS     152
VT     125
MT     104
AK      98
NB      97
SD      75
ND      67
WY      62
NL      58
PE      20
YT      10
NT       7
NU       1
Name: count, dtype: int64

In [82]:
# turn value_counts into pandas df
counts_by_region = counts_by_region.to_frame()


In [83]:
counts_by_region.head()

,count
region,
CA,8578
NY,7056
TX,3540
FL,3522
ON,2833


In [84]:
pd.set_option('display.max_rows', None)
counts_by_region


,count
region,
CA,8578
NY,7056
TX,3540
FL,3522
ON,2833
PA,2693
NJ,2407
IL,2182
GA,1818


In [ ]:
# done

fixregion_monk = chinese_restaurants[chinese_restaurants['region'] == 'Monk']
fixregion_monk

,fsq_place_id,name,latitude,longitude,address,locality,region,postcode,admin_region,post_town,...,email,facebook_id,instagram,twitter,fsq_category_ids,fsq_category_labels,placemaker_url,unresolved_flags,geom,bbox
12592,53655355498ebeabc3bfc92c,Chuan Saveur Masion,45.45698,-73.593145,2110 Rue Jolicoeur,Montreal,Monk,H4E 1X6,NaN,NaN,...,NaN,NaN,NaN,NaN,['52af3b773cf9994f4e043c03'],['Dining and Drinking > Restaurant > Asian Res...,https://foursquare.com/placemakers/review-plac...,NaN,b'\x00\x00\x00\x00\x01\xc0Re\xf6\x15VXx@F\xba~...,"{'xmin': -73.59314473564325, 'ymin': 45.456979..."


In [ ]:
# done
# display all rows where 'region' is 'ケベック', '安大略省', '安大略', 'Mexico', or 'New York'
fixregion_misc = chinese_restaurants[chinese_restaurants['region'].isin(['ケベック', '安大略省', '安大略', 'Mexico', 'New York'])]
fixregion_misc


,fsq_place_id,name,latitude,longitude,address,locality,region,postcode,admin_region,post_town,...,email,facebook_id,instagram,twitter,fsq_category_ids,fsq_category_labels,placemaker_url,unresolved_flags,geom,bbox
12445,4e2eec2ad4c058fdbeee58ab,FORTUNE 2004,46.810879,-71.215122,NaN,ケベック,ケベック,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,['4bf58dd8d48988d145941735'],['Dining and Drinking > Restaurant > Asian Res...,https://foursquare.com/placemakers/review-plac...,NaN,b'\x00\x00\x00\x00\x01\xc0Q\xcd\xc4\x8c\xfb=\x...,"{'xmin': -71.21512150321217, 'ymin': 46.810878..."
12569,51729107498ebf7fc424ef49,Sensai,45.447928,-73.572102,Ocean Coral H10 Resort,Riviera Maya,Mexico,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,['4bf58dd8d48988d145941735'],['Dining and Drinking > Restaurant > Asian Res...,https://foursquare.com/placemakers/review-plac...,NaN,b'\x00\x00\x00\x00\x01\xc0Rd\x9dP\x08\x82\xbb@...,"{'xmin': -73.57210160094375, 'ymin': 45.447927..."
27764,4e63fc62c65b2dc8a04cb2fa,Fu Kee Chinese Cuisine,42.980028,-81.211768,601 Hamilton Rd,伦敦,安大略省,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,['4bf58dd8d48988d145941735'],['Dining and Drinking > Restaurant > Asian Res...,https://foursquare.com/placemakers/review-plac...,NaN,b'\x00\x00\x00\x00\x01\xc0TM\x8d\x9dF\xe8(@E}q...,"{'xmin': -81.21176845478533, 'ymin': 42.980028..."
50767,4c84d07b2f1c236ac7f74e43,"Happy Lamb Hot Pot, Flushing 快乐小羊",40.762194,-73.828995,136-59 37th Avenue,Queens,New York,11354,NaN,NaN,...,NaN,NaN,NaN,NaN,['4bf58dd8d48988d145941735'],['Dining and Drinking > Restaurant > Asian Res...,https://foursquare.com/placemakers/review-plac...,NaN,b'\x00\x00\x00\x00\x01\xc0Ru\x0eA\x0bc\x0b@Da\...,"{'xmin': -73.828995, 'ymin': 40.762194, 'xmax'..."
58243,588d327a4988da585259114f,Old Beijing BBQ,43.784412,-79.277670,NaN,Toronto,安大略,M1S,NaN,NaN,...,NaN,NaN,NaN,NaN,['52af3b463cf9994f4e043bfe'],['Dining and Drinking > Restaurant > Asian Res...,https://foursquare.com/placemakers/review-plac...,NaN,b'\x00\x00\x00\x00\x01\xc0S\xd1\xc5Y\xd3\xcb\x...,"{'xmin': -79.27767034228535, 'ymin': 43.784411..."


In [ ]:
# done
# delete rows where index is 12445, 12569, 27764
chinese_restaurants = chinese_restaurants.drop(index=[12445, 12569, 27764])

In [ ]:
# done
chinese_restaurants.loc[chinese_restaurants['region'] == 'New York', 'region'] = 'NY'

In [ ]:
# done
chinese_restaurants.loc[chinese_restaurants['region'] == '安大略', 'region'] = 'ON'


In [ ]:
# done

chinese_restaurants.loc[chinese_restaurants['region'] == 'Monk', 'region'] = 'QC'

In [ ]:
# done
# correct misspellings in 'region'

chinese_restaurants.loc[chinese_restaurants['region'] == 'Ontario', 'region'] = 'ON'
chinese_restaurants.loc[chinese_restaurants['region'] == 'British Columbia', 'region'] = 'BC'
chinese_restaurants.loc[chinese_restaurants['region'] == 'Saskatchewan', 'region'] = 'SK'
chinese_restaurants.loc[chinese_restaurants['region'] == 'Calif', 'region'] = 'CA'
chinese_restaurants.loc[chinese_restaurants['region'] == 'Alberta', 'region'] = 'AB'
chinese_restaurants.loc[chinese_restaurants['region'] == 'On', 'region'] = 'ON'
chinese_restaurants.loc[chinese_restaurants['region'] == 'Saakatchewan', 'region'] = 'SK'
chinese_restaurants.loc[chinese_restaurants['region'] == 'On.', 'region'] = 'ON'
chinese_restaurants.loc[chinese_restaurants['region'] == 'Québec', 'region'] = 'QC'
chinese_restaurants.loc[chinese_restaurants['region'] == 'BC Columbia', 'region'] = 'BC'
chinese_restaurants.loc[chinese_restaurants['region'] == 'Bc', 'region'] = 'BC'
chinese_restaurants.loc[chinese_restaurants['region'] == 'M1G 1P9', 'region'] = 'ON'
chinese_restaurants.loc[chinese_restaurants['region'] == 'Onrtario', 'region'] = 'ON'

In [ ]:
# done
# display all rows where 'region' is 'Canada'
fixregion_canada = chinese_restaurants[chinese_restaurants['region'] == 'Canada']
fixregion_canada



,fsq_place_id,name,latitude,longitude,address,locality,region,postcode,admin_region,post_town,...,email,facebook_id,instagram,twitter,fsq_category_ids,fsq_category_labels,placemaker_url,unresolved_flags,geom,bbox
15953,4bcfe1da462cb7130195d707,Terracotta - Modern Chinese,49.283480,-123.103001,52 Alexander St,Vancouver,Canada,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,['4bf58dd8d48988d145941735'],['Dining and Drinking > Restaurant > Asian Res...,https://foursquare.com/placemakers/review-plac...,NaN,b'\x00\x00\x00\x00\x01\xc0^\xc6\x97\x91PNZ@H\x...,"{'xmin': -123.10300095407857, 'ymin': 49.28348..."
16107,4b04a6fbf964a520f35522e3,Chop & Wok,49.265765,-123.121365,786 W. 6th Ave.,Vancouver,Canada,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,['4bf58dd8d48988d145941735'],['Dining and Drinking > Restaurant > Asian Res...,https://foursquare.com/placemakers/review-plac...,['closed'],b'\x00\x00\x00\x00\x01\xc0^\xc7\xc4r\xa37&@H\x...,"{'xmin': -123.12136522234923, 'ymin': 49.26576..."
16306,4aec8791f964a52053c821e3,Cafe seventh heaven,49.183088,-123.133930,"2792 aberdeen centre, 4151 hazelbridge",Richmond,Canada,V6x 4j7,NaN,NaN,...,NaN,NaN,NaN,NaN,['4bf58dd8d48988d145941735'],['Dining and Drinking > Restaurant > Asian Res...,https://foursquare.com/placemakers/review-plac...,NaN,b'\x00\x00\x00\x00\x01\xc0^\xc8\x92NX*@@H\x97o...,"{'xmin': -123.13392981157176, 'ymin': 49.18308..."


In [ ]:
# done
# indices 15953, 16107, and 16306 are in BC, change 'region'
chinese_restaurants.loc[[15953, 16107, 16306], 'region'] = 'BC'

In [ ]:
# pd.reset_option('display.max_rows')